In [6]:
import pandas as pd 
import os,sys
import timeit

### Read in germline2 .match file, convert to dict of valid entries (pairs with >=1 segment that's 5cM or more)

In [15]:
matchfile = '/data100t1/home/grahame/projects/compadre/unified-simulations/analysis-output/simulations/AMR/uniform3_size20_sim100/uniform3_size20_sim100/ss/ersa_input_final.txt'

segment_dict = {}

with open (matchfile, 'r') as f:
    for line in f:
        ls = line.split('\t')
        id1, id2 = ls[0], ls[1]
        cmlen = round(float(ls[4]), 2)
        key = f"{id1}:{id2}"
        if cmlen >= 5.0:
            if key not in segment_dict:
                segment_dict[key] = (cmlen,)
            else:
                segment_dict[key] += (cmlen,)

In [52]:
# Dict with segments as keys

segment_dict_larger_1 = {}

with open (matchfile, 'r') as f:
    for line in f:
        ls = line.split('\t')
        id1, id2, start, end, cmlen, chrom = ls[0], ls[1], int(ls[2]), int(ls[3]), round(float(ls[4]), 2), int(ls[5].strip())
        key = f"{chrom}:{start}:{end}:{cmlen}"
        value = f"{id1}:{id2}"
        if cmlen >= 5.0:
            if key not in segment_dict_larger_1:
                segment_dict_larger_1[key] = (value,)
            else:
                segment_dict_larger_1[key] += (value,)

In [43]:
# Dict with ids as keys

segment_dict_larger_2 = {}

with open (matchfile, 'r') as f:
    for line in f:
        ls = line.split('\t')
        id1, id2, start, end, cmlen, chrom = ls[0], ls[1], int(ls[2]), int(ls[3]), round(float(ls[4]), 2), int(ls[5].strip())
        key = f"{id1}:{id2}"
        value = f"{chrom}:{start}:{end}:{cmlen}"
        if cmlen >= 5.0:
            if key not in segment_dict_larger_2:
                segment_dict_larger_2[key] = (value,)
            else:
                segment_dict_larger_2[key] += (value,)

In [59]:
# Dict with ids as keys and tuple values 

segment_dict_larger_3 = {}

with open (matchfile, 'r') as f:
    for line in f:
        ls = line.split('\t')
        id1, id2, start, end, cmlen, chrom = ls[0], ls[1], int(ls[2]), int(ls[3]), round(float(ls[4]), 2), int(ls[5].strip())
        key = f"{id1}:{id2}"
        value = (chrom, start, end, cmlen)
        if cmlen >= 5.0:
            if key not in segment_dict_larger_3:
                segment_dict_larger_3[key] = (value,)
            else:
                segment_dict_larger_3[key] += (value,)

### Now we have to figure out how to update the ERSA iteration to handle this data structure instead of the large file

In [41]:
# In-memory footprint comparison with a dataframe

df = pd.read_csv(matchfile, sep='\t', header=None)

dict_size = sys.getsizeof(segment_dict_larger_1) + sum(sys.getsizeof(k) + sys.getsizeof(v) for k, v in segment_dict_larger_1.items())
df_size = df.memory_usage(deep=True).sum()
difference_multiplier = int(df_size / dict_size)

print(f"Dictionary: {dict_size} bytes\nDataFrame: {df_size} bytes ({difference_multiplier}x difference)")

Dictionary: 198702 bytes
DataFrame: 795200 bytes (4x difference)


In [61]:
# Compare larger dict sizes 

dict_size1 = sys.getsizeof(segment_dict_larger_1) + sum(sys.getsizeof(k) + sys.getsizeof(v) for k, v in segment_dict_larger_1.items())
dict_size2 = sys.getsizeof(segment_dict_larger_2) + sum(sys.getsizeof(k) + sys.getsizeof(v) for k, v in segment_dict_larger_2.items())
dict_size3 = sys.getsizeof(segment_dict_larger_3) + sum(sys.getsizeof(k) + sys.getsizeof(v) for k, v in segment_dict_larger_3.items())

print (f"Dictionary (Segment keys): {dict_size1} bytes\nDictionary (ID pair keys, concat str values): {dict_size2} bytes\nDictionary (ID pair keys, tuple values): {dict_size3} bytes")

Dictionary (Segment keys): 198702 bytes
Dictionary (ID pair keys, concat str values): 53914 bytes
Dictionary (ID pair keys, tuple values): 53914 bytes


### The smallest option is using id1:id2 as the key and a tuple of segment information as the value

In [58]:
segment_dict_larger_3['id2:id10']

((20, 1945629, 4586645, 6.2),)

### Build 1KG superpopulation 'lists' to match HH's downloaded data, and validate by comparing EUR list to his ids

In [ ]:
hh_eur_id_file = '/data100t1/share/reference_data/1KG/GRCh38/EUR/1KG.EUR.ids'
hh_eur_ids = []
with open (hh_eur_id_file, 'r') as f:
    for line in f:
        hh_eur_ids.append(line.strip())

hh_amr_id_file = '/data100t1/share/reference_data/1KG/GRCh38/AMR/1KG.AMR.ids'
hh_amr_ids = []
with open (hh_amr_id_file, 'r') as f:
    for line in f:
        hh_amr_ids.append(line.strip())

 
id_info = pd.read_csv('/data100t1/home/grahame/projects/compadre/unified-simulations/data/igsr-1000 genomes 30x on grch38.tsv.tsv', sep='\t')

eur_ids = id_info[id_info['Superpopulation code'] == 'EUR']['Sample name'].to_list()
amr_ids = id_info[id_info['Superpopulation code'] == 'AMR']['Sample name'].to_list()
afr_ids = id_info[id_info['Superpopulation code'] == 'AFR']['Sample name'].to_list()

In [ ]:
other_test = pd.read_csv('/data100t1/home/grahame/projects/compadre/unified-simulations/data/other.tsv', sep='\t', usecols=['SAMPLE_NAME', 'POPULATION'])

other_ids = other_test['SAMPLE_NAME'].to_list()

In [ ]:
other_eur = [x for x in other_ids if x in eur_ids]
other_afr = [x for x in other_ids if x in afr_ids]
other_amr = [x for x in other_ids if x in amr_ids]

In [ ]:
with open('/data100t1/share/reference_data/1KG/GRCh38/AFR/1KG.AFR.ids', 'w+') as outfile:
    for x in other_afr:
        outfile.write(x+'\n')

### concatenate all my unrelated subpop ids into a single list (for fixing the unrelated plink-binary set)

In [ ]:
subpop_folder = '/data100t1/home/grahame/projects/compadre/primus/primus_old_1KG/lib/1KG_reference_data/1KG_unrelateds_by_subpop'

subpop_files = [f'{subpop_folder}/{f.name}' for f in os.scandir(subpop_folder)]

master_list = []

for f in subpop_files:
    df = pd.read_csv(f, sep='\t')
    ids = df['IID'].to_list()
    master_list.extend(ids)

In [ ]:
len(master_list)

2340

In [ ]:
with open(f'{subpop_folder}/1KG_formatted_set_all', 'w+') as outfile:
    for x in master_list:
        outfile.write(x+'\n')

### replace duplicate FIDs in the all_unrelateds_NEW plink fileset with the corresponding subpop

In [ ]:
famfile = '/data100t1/home/grahame/projects/compadre/primus/primus_old_1KG/lib/1KG_reference_data/all_unrelateds_NEW.fam'
famfile2 = '/data100t1/home/grahame/projects/compadre/primus/primus_old_1KG/lib/1KG_reference_data/all_unrelateds_NEW.fam2'

subpop_folder = '/data100t1/home/grahame/projects/compadre/primus/primus_old_1KG/lib/1KG_reference_data/1KG_unrelateds_by_subpop'

subpop_files = [f'{subpop_folder}/{f.name}' for f in os.scandir(subpop_folder) if not f.name.endswith('all')]

subpop_dict = {}

for f in subpop_files:
    pop = f.split('_')[-1]
    df = pd.read_csv(f, sep='\t')
    ids = df['IID'].to_list()
    for x in ids:
        subpop_dict[x] = pop

In [ ]:
with open (famfile, 'r') as infile, open (famfile2, 'w+') as outfile:
    for line in infile:
        ls = line.split(' ')
        iid = ls[1]
        popmatch = subpop_dict[iid]
        outfile.write(f'{popmatch} {iid} 0 0 0 -9\n')